## Zadanie domowe - Algorytm Canny'ego

Celem zadania domowego jest wykonanie pełnej implementacji algorytmu Canny'ego.

W ramach ćwiczenia w trakcie laboratorium wyznaczono obrazy $g_{NH}$ i $g_{NL}$.
Dla przypomnienia:
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Wykorzystuje się więc krawędzie "potencjalne", aby uzupełnić nieciągłości.
Procedura wygląda następująco:
1. Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
2. Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
3. Stwórz obraz, który zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
4. Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź, czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź.
        Można to wykonać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.
8. Porównaj wynik algorytmu z wynikiem OpenCV.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować w bardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj pełny algorytm Canny'ego.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

def canny_complete(img,size,th,tl):

    kernel =np.ones((size,size))/(size*size)
    new_image = cv2.filter2D(img, -1, kernel)

    S1=np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
    S2=np.array([[-1,-2,-1],[0,0,0],[1,2,1]])
    Gx=cv2.filter2D(new_image, -1, S1)
    Gy=cv2.filter2D(new_image, -1, S2)

    M = np.sqrt(Gx*Gx + Gy*Gy)
    alpha_rad = np.arctan2(Gx, Gy)
    alpha_deg = np.degrees(alpha_rad)

    J,K = alpha_deg.shape
    for x in range(J):
        for y in range(K):
            if alpha_deg[x][y] < 0:
                alpha_deg[x][y] + 180
                
    X,Y = img.shape
    direct = np.zeros((X,Y))

    for i in range (X):
        for j in range (Y):
            if (0 <= alpha_deg[i][j] < 22.5):
                direct[i][j]=0
            elif (22.5 <= alpha_deg[i][j] < 67.5):
                direct[i][j]=45
            elif (67.5 <= alpha_deg[i][j] < 112.5):
                direct[i][j]=90
            elif (112.5 <= alpha_deg[i][j] < 157.5):
                direct[i][j]=135
            elif (157.5 <= alpha_deg[i][j] <= 180):
                direct[i][j]=0

    gn = nonmax(direct, M)
    gnh=gn>=th
    gnl = np.zeros((X,Y))
    for i in range(X):
        for j in range(Y):
            if (gn[i][j]<th and gn[i][j] >=tl):
                gnl[i][j] = 1
    list_gnh = []
    for i in range(X):
        for j in range(Y):
            if gnh[i][j] == 1:
                list_gnh.append([i,j])

    visited = np.zeros((X,Y))
    edges  = np.zeros((X,Y))
    while list_gnh:
        el = list_gnh.pop()
        if visited[el[0]][el[1]] == 0:
            visited[el[0]][el[1]] = 1
            edges[el[0]][el[1]] = 1
            if el[0] != 0 and el[1] != 0:
                for i in range(el[0]-1,el[0]+2):
                    for j in range(el[1]-1,el[1]+2):
                        if(gnl[i,j]==1):
                            list_gnh.append([i,j])            
    return edges,gnh,gnl

def nonmax(direct,M):
    X,Y = direct.shape
    gn=np.zeros((X,Y))
    
    for i in range (1,X-1):
        for j in range (1,Y-1):
            if(direct[i][j]==0):    #poziom
                if(M[i][j-1]>M[i,j] or M[i][j+1]>M[i][j]):
                    gn[i][j]=0
                else:
                    gn[i,j]=M[i,j]
            elif(direct[i][j]==45):     #skos prawy 
                if(M[i+1][j-1]>M[i,j] or M[i-1][j+1]>M[i,j]):
                    gn[i][j]=0
                else:
                    gn[i,j]=M[i,j]
            elif(direct[i,j]==90):  #pion
                if(M[i+1][j]>M[i,j] or M[i-1][j]>M[i][j]):
                    gn[i,j]=0
                else:
                    gn[i,j]=M[i,j]
            elif(direct[i,j]==135):   # skos lewy 
                if(M[i-1][j-1]>M[i,j] or M[i+1][j+1]>M[i][j]):
                    gn[i,j]=0
                else:
                    gn[i,j]=M[i,j]
    return gn





In [ ]:
house = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)
edges, gnh,gnl = canny_complete(house,3,6,12)

house_opencv=cv2.Canny(house,6,11,None,3,1)


plt.imshow(house, 'gray')
plt.title('Obraz oryginalny')
plt.axis('off')
plt.show()

plt.imshow(gnh, 'gray')
plt.title('GNH')
plt.axis('off')
plt.show()



plt.imshow(edges, 'gray')
plt.title('Obraz wynikowy')
plt.axis('off')
plt.show()

plt.imshow(house_opencv, 'gray')
plt.title('Obraz wynikowy z funkcji wbudowanej ')
plt.axis('off')
plt.show()







Na obraziez funkcji wbudowanej bardziej widoczne białe plamy w górnej częsci obrazu oraz dachówki. Na obu wynikach słabo widoczne okna z lewej strony.